# Trend Screener

In [113]:
pip install "yfinance[nospam]" stocktrends matplotlib pandas requests_cache

Note: you may need to restart the kernel to use updated packages.


In [114]:
import yfinance as yf
import pandas as pd
import stocktrends as st
import numpy as np
import requests_cache

session = requests_cache.CachedSession('yfinance.cache')
session.headers['User-agent'] = 'Mozilla/5.0 (compatible; MSIE 7.0; Windows; U; Windows NT 10.4; Win64; x64; en-US Trident/4.0)'

results = []

tickers_txt_path = 'us_stock2.txt'

try:
    with open(tickers_txt_path, 'r') as file:
        tickers = [line.strip() for line in file if line.strip()]
except Exception as e:
    print(f"Error reading tickers from text file: {e}")
    tickers = []

start = '2023-03-26'
end = '2024-07-26'

data = yf.download(tickers, start = start, end = end, session=session)

print(data)

[*********************100%%**********************]  118 of 118 completed

16 Failed downloads:
['ACAXR', 'AAIC', 'ABCM', 'ACAXU', 'AAIN', 'ACAH', 'ACRX', 'AAU', 'ACRO', 'ACAX', 'ACOR', 'ACDCW']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
['ABLLW', 'ACER', 'ACONW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
['ACBA']: YFChartError('%ticker%: No data found, symbol may be delisted')


Price       Adj Close                                                     \
Ticker             AA  AACG   AACI  AACIU    AACT  AADI AAIC AAIN    AAL   
Date                                                                       
2023-03-27  40.119999  1.82  10.07  10.00     NaN  7.25  NaN  NaN  13.94   
2023-03-28  40.529999  1.81  10.07  10.00     NaN  6.85  NaN  NaN  13.98   
2023-03-29  40.939999  1.83  10.07  10.00     NaN  7.12  NaN  NaN  14.35   
2023-03-30  41.759998  1.87  10.08  10.00     NaN  7.11  NaN  NaN  14.47   
2023-03-31  42.560001  1.93  10.08  10.00     NaN  7.24  NaN  NaN  14.75   
...               ...   ...    ...    ...     ...   ...  ...  ...    ...   
2024-07-19  34.919998  0.65  11.39  11.93  10.700  1.43  NaN  NaN  10.58   
2024-07-22  34.529999  0.65  11.35  11.93  10.700  1.51  NaN  NaN  10.69   
2024-07-23  33.900002  0.67  11.30  11.93  10.715  1.57  NaN  NaN  10.48   
2024-07-24  32.840000  0.68  11.35  11.09  10.720  1.57  NaN  NaN  10.17   
2024-07-25  

In [118]:
def ehlers_instantaneous_trendline(source, alpha=0.07):
    itrend = np.zeros_like(source)
    trigger = np.zeros_like(source)

    for i in range(len(source)):
        if i < 7:
            itrend[i] = (source.iloc[i] + (2 * source.iloc[i - 1]) + source.iloc[i - 2]) / 4
        else:
            itrend[i] = (
                ((alpha - (alpha ** 2 / 4)) * source.iloc[i]) +
                (0.5 * alpha ** 2 * source.iloc[i - 1]) -
                ((alpha - (0.75 * alpha ** 2)) * source.iloc[i - 2]) +
                (2 * (1 - alpha) * itrend[i - 1]) -
                ((1 - alpha) ** 2 * itrend[i - 2])
            )

        if i >= 2:
            trigger[i] = (2 * itrend[i]) - itrend[i - 2]

    return itrend

def calculate_trend(df, ticker):
    df[f'{ticker} 52W_High'] = df['High'].rolling(window=252).max()
    df[f'{ticker} 52W_High_Ratio'] = df['High'] / df[f'{ticker} 52W_High']
    df[f'{ticker} 52W_ROC'] = (df['High'] - df['High'].shift(252)) / df['High'].shift(252)
    
    df[f'{ticker} 52W_Low'] = df['Low'].rolling(window=252).min()
    df[f'{ticker} 52W_Low_Ratio'] = df['Low'] / df[f'{ticker} 52W_Low']

    df[f'{ticker} 50MA'] = df['Adj Close'].rolling(window=50).mean()
    df[f'{ticker} 150MA'] = df['Adj Close'].rolling(window=150).mean()
    df[f'{ticker} 200MA'] = df['Adj Close'].rolling(window=200).mean()

    df[f'{ticker} MA_Uptrend'] = (
        (df['Adj Close'] > df[f'{ticker} 50MA']) &
        (df[f'{ticker} 50MA'] > df[f'{ticker} 150MA']) &
        (df[f'{ticker} 150MA'] > df[f'{ticker} 200MA']) &
        (df[f'{ticker} 50MA'].diff() > 0) &
        (df[f'{ticker} 150MA'].diff() > 0) &
        (df[f'{ticker} 200MA'].diff() > 0)
    )

    df[f'{ticker} 200MA_Uptrend'] = df[f'{ticker} 200MA'].diff() > 0
    df[f'{ticker} 200MA_1M_Uptrend'] = df[f'{ticker} 200MA_Uptrend'].rolling(window=20).mean() * 100
    df[f'{ticker} 200MA_4M_Uptrend'] = df[f'{ticker} 200MA_Uptrend'].rolling(window=80).mean() * 100

    itrend = ehlers_instantaneous_trendline(df['Adj Close'])
    df[f'{ticker} ITrend'] = itrend
    df[f'{ticker} ITrend_Uptrend'] = df[f'{ticker} ITrend'].diff() > 0

    df[f'{ticker} Stop_Loss'] = df[f'{ticker} 50MA'] / df['Adj Close']
    df[f'{ticker} g'] = df[f'{ticker} ITrend'] / df[f'{ticker} ITrend'].shift(1)
    df[f'{ticker} Risk_Reward'] = df[f'{ticker} g'] / df[f'{ticker} Stop_Loss']

    return df

results = {}
for ticker in tickers:
    try:
        ticker_df = pd.DataFrame( data['Adj Close'][ticker])
        ticker_df['High'] = data['High'][ticker]
        ticker_df['Low'] = data['Low'][ticker]
        ticker_df = ticker_df.rename(columns={ticker: 'Adj Close'})
        ticker_df = calculate_trend(ticker_df, ticker)

        results[ticker] = ticker_df
    except Exception as e:
        print(f"Error calculating trend for {ticker}: {e}")

roc_data = pd.DataFrame({ticker: df[f'{ticker} 52W_ROC'] for ticker, df in results.items()})
roc_ranks = roc_data.rank(axis=1)
roc_ranks = roc_ranks.div(len(roc_ranks.columns)).rename(columns={ticker: f'{ticker} RS' for ticker in tickers})

for ticker in tickers:
    try:
        results[ticker] = results[ticker].join(roc_ranks[f'{ticker} RS'])
        results[ticker][f'{ticker} RS_13W'] = results[ticker][f'{ticker} RS'].rolling(window=65).mean()
    except Exception as e:
        print(f"Error adding RS for {ticker}: {e}")


for ticker, res_df in results.items():
    print(f"\n{ticker} Moving Averages and Uptrend Status:\n", res_df.tail())

summary_data = {}
for ticker, df in results.items():
    latest_data = df.iloc[-1]
    uptrend = latest_data[f'{ticker} 52W_High_Ratio'] >= 0.75 and \
        latest_data[f'{ticker} 52W_Low_Ratio'] >= 1.25 and \
        latest_data[f'{ticker} MA_Uptrend'] and \
        latest_data[f'{ticker} ITrend_Uptrend'] and \
        latest_data[f'{ticker} g'] > 1.01 and \
        latest_data[f'{ticker} RS'] > 0.7

    if uptrend:
        summary_data[ticker] = {
            '52W_High_Ratio': round(latest_data[f'{ticker} 52W_High_Ratio'], 2),
            '52W_Low_Ratio': round(latest_data[f'{ticker} 52W_Low_Ratio'], 2),
            '52W_ROC': round(latest_data[f'{ticker} 52W_ROC'], 2),
            'RS': latest_data[f'{ticker} RS'],
            'RS_13W': latest_data[f'{ticker} RS_13W'],
            'Stop_Loss': round(latest_data[f'{ticker} Stop_Loss'], 2),
            'g': round(latest_data[f'{ticker} g'], 4),
            'Risk_Reward': round(latest_data[f'{ticker} Risk_Reward'], 2),
        }

summary_df = pd.DataFrame(summary_data).T
summary_df.to_csv('filtered_stocks.csv')

print(summary_df)


EBS Moving Averages and Uptrend Status:
             Adj Close   High    Low  EBS 52W_High  EBS 52W_High_Ratio  \
Date                                                                    
2024-07-19      11.20  11.97  11.04         12.74            0.939560   
2024-07-22      11.46  11.51  10.61         12.74            0.903454   
2024-07-23      12.49  12.63  11.47         12.74            0.991366   
2024-07-24      13.81  13.97  11.93         13.97            1.000000   
2024-07-25      13.80  14.61  13.50         14.61            1.000000   

            EBS 52W_ROC  EBS 52W_Low  EBS 52W_Low_Ratio  EBS 50MA  EBS 150MA  \
Date                                                                           
2024-07-19     0.626359         1.42           7.774648    6.8400   3.813733   
2024-07-22     0.576712         1.42           7.471831    6.9810   3.874867   
2024-07-23     0.592686         1.42           8.077465    7.1434   3.941267   
2024-07-24     0.811933         1.42          